In [1]:
!pip install -Uqq sentence-transformers nltk pypdf langchain_community

In [2]:
!pip install -U rapidocr-onnxruntime

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.11.8-py3.12.egg is deprecated. pip 

In [11]:
from sentence_transformers import SentenceTransformer, util
import torch
# from nltk.tokenize import sent_tokenize
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.parsers import RapidOCRBlobParser
# from pypdf import PdfReader

model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2", device='cuda')

loader = PyPDFLoader('BERTforIR.pdf')

docs = loader.load()


splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)


query = "How is BERT used for Information Retrieval?"

q_embeddings = model.encode(query, convert_to_tensor=True)
embeddings = model.encode(chunks, convert_to_tensor=True)

sim = util.euclidean_sim(q_embeddings, embeddings)[0]
k = 10
top_results = torch.topk(sim, k=k)

for score, idx in zip(top_results.values, top_results.indices):
    print(f"{chunks[idx]} (Score: {score.item():.4f})", end = '\n\n')


TypeError: 'Document' object is not subscriptable

In [94]:
chunks[util.semantic_search(q_embeddings, embeddings)[0][2]['corpus_id']]

'This enables the reader to understand BERT-related models from\nvarious perspectives in a step-by-step manner and discover new research opportunities by following this organization.'

In [96]:
help(util.cos_sim)

Help on function cos_sim in module sentence_transformers.util:

cos_sim(a: 'list | np.ndarray | Tensor', b: 'list | np.ndarray | Tensor') -> 'Tensor'
    Computes the cosine similarity between two tensors.

    Args:
        a (Union[list, np.ndarray, Tensor]): The first tensor.
        b (Union[list, np.ndarray, Tensor]): The second tensor.

    Returns:
        Tensor: Matrix with res[i][j] = cos_sim(a[i], b[j])

